# Experiment data analysis

In [180]:
import numpy
from scipy.special import binom

In [181]:
experiment_colors = ["red", "blue", "green", "pink", "gray", "yellow", "brown", "black"]


def is_negatively_associated_color(color):
    assert isinstance(color, str) and color in experiment_colors
    return color == "black"


def is_positively_associated_color(color):
    assert isinstance(color, str) and color in experiment_colors
    return color in {"red", "blue", "green", "pink", "yellow"}

## Results of the experiment are here

Current data is randomly generated, later insert something like
```
results_control = [1,2,5,2,7,2,3,4]
```
where the numbers correspond to how many kids in a specific group has picked some color (order as in `experiment_colors`).

In [194]:
def random_data(): return [10+round(i) for i in numpy.random.uniform(0,3, size=len(experiment_colors))]

results_control = random_data()
results_positive_story = random_data()
results_positive_story[experiment_colors.index('green')] += 40
results_negative_story = random_data()
results_negative_story[experiment_colors.index('black')] += 10

for group_results in [results_control, results_positive_story, results_negative_story]:
    assert len(group_results) == len(experiment_colors)
    assert all(isinstance(x, int) and x >= 0 for x in group_results)

## Hypothesis testing
We first compute the expected distribution from the control group data. Our null hypothesis is that the data collected from the positive/negative story groups follows this distribution.

Our alternative hypothesis is that the data collected from (non-control) children follows from distributions
where probability of colors correlating with their standard biases is *higher* than in the control group.

We then calculate the probability of witnessing collected data under the assumption that the null hypothesis holds.
If this probability is very small (that is, below α = 0.05) we assume that the null hypothesis is
unlikely and therefore our alternative hypothesis seems correct.

In [195]:
def bernoulli_p(group_results, color_association_predicate):
    return sum(
        n
        for color_idx, n in enumerate(group_results)
        if color_association_predicate(experiment_colors[color_idx])
    ) / sum(group_results)

In [196]:
positive_control_p = bernoulli_p(results_control, is_positively_associated_color)
negative_control_p = bernoulli_p(results_control, is_negatively_associated_color)

In [197]:
def binomial_pdf(p, trials, successes):
    return binom(trials, successes) * p**successes * (1 - p)**(trials - successes)

In [198]:
def probability_of_results_under_null_hypothesis(null_p, group_results, color_association_predicate):
    prob = 0
    for successes in range(
            sum(
                n
                for color_idx, n in enumerate(group_results)
                if color_association_predicate(experiment_colors[color_idx])),
            sum(group_results)):
        prob += binomial_pdf(null_p, sum(group_results), successes)
    return prob

### Final results
If the following probabilities are under α=0.05, then it can be said that we have rejected the null hypothesis
and therefore proven that it's likely the expected biases exist.

In [199]:
α = 0.05

In [200]:
pp = probability_of_results_under_null_hypothesis(
        positive_control_p,
        results_positive_story,
        is_positively_associated_color)
pp, pp < α

(0.011886094750535226, True)

In [201]:
pn = probability_of_results_under_null_hypothesis(
        negative_control_p,
        results_negative_story,
        is_negatively_associated_color)
pn, pn < α

(0.006836533411551267, True)

Probability of observing the collected data under the null hypothesis is below α, we therefore assume
that there is a statistically significant correlation we were after.

## Notes
* The code currently does not take children age into account, I assume that two age groups are separate experiments
* If you want to test each color, replace predicates with something like `lambda c: c == 'red'`